In [1]:
#Imports
import time
import os
import platform
import json
from datetime import datetime
from typing import Any, Optional, Dict, List
import random
import pandas as pd

In [2]:
class NoCircular:
    """Classe que representa um nó da lista circular"""
    def __init__(self, data: Any):
        self.data = data
        self.next: Optional['NoCircular'] = None

'''OBS: Os métodos com terminação **_interno** são chamados sempre pelas funções feitas para medição/salvamento do tempo de execução.
 exemplo do método que insere no início da lista:

 def inserir_inicio(self, data: Any):
        return self._medir_tempo("inserir_inicio", self._inserir_inicio_interno, data)
 '''

class ListaCircular:
    """
    Lista circular onde o último nó aponta de volta para o primeiro,
    formando um ciclo. Mantém ponteiros para head e tail para otimização.
    """

    def __init__(self):
        self.head: Optional[NoCircular] = None
        self.tail: Optional[NoCircular] = None  # Ponteiro para o último nó
        self.size = 0
        self.tempos_execucao: List[Dict] = []

        # Criar pasta para salvar os tempos de execução
        self.pasta_tempos = "tempos_de_execucao"
        if not os.path.exists(self.pasta_tempos):
            os.makedirs(self.pasta_tempos)

    def _medir_tempo(self, operacao: str, func, *args, **kwargs):
        """Mede o tempo de execução de uma operação e salva o resultado"""
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fim = time.time()

        tempo_execucao = fim - inicio

        # Registrar o tempo de execução
        registro = {
            'operacao': operacao,
            'tempo_execucao_ms': tempo_execucao * 1000,  # Converter para milissegundos
            'timestamp': datetime.now().isoformat(),
            'sistema_operacional': platform.system(),
            'versao_so': platform.release(),
            'arquitetura': platform.machine(),
            'tamanho_lista': self.size
        }

        self.tempos_execucao.append(registro)
        self._salvar_tempo(registro)

        return resultado

    def _salvar_tempo(self, registro: Dict):
        """Salva o tempo de execução em arquivo específico do SO"""
        sistema = platform.system()
        nome_arquivo = f"lista_encadeada_circular_{sistema.lower()}_tempos_execucao.json"
        caminho_arquivo = os.path.join(self.pasta_tempos, nome_arquivo)

        # Carregar dados existentes ou criar nova lista
        dados = []
        if os.path.exists(caminho_arquivo):
            try:
                with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                    dados = json.load(f)
            except (json.JSONDecodeError, FileNotFoundError):
                dados = []

        # Adicionar novo registro
        dados.append(registro)

        # Salvar dados atualizados
        with open(caminho_arquivo, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)

    def _inserir_inicio_interno(self, data: Any):
        """Método interno para inserir no início"""
        novo_no = NoCircular(data)
        if self.head is None:
            # Lista vazia - primeiro elemento
            self.head = novo_no
            self.tail = novo_no
            novo_no.next = novo_no  # Aponta para si mesmo (circular)
        else:
            # Inserir no início e manter circularidade
            novo_no.next = self.head
            self.tail.next = novo_no  # O último nó agora aponta para o novo primeiro
            self.head = novo_no
        self.size += 1
        return True

    def _inserir_fim_interno(self, data: Any):
        """Método interno para inserir no fim (otimizado)"""
        novo_no = NoCircular(data)
        if self.head is None:
            # Lista vazia - primeiro elemento
            self.head = novo_no
            self.tail = novo_no
            novo_no.next = novo_no  # Aponta para si mesmo (circular)
        else:
            # Inserir no fim e manter circularidade
            novo_no.next = self.head  # Novo último aponta para o primeiro
            self.tail.next = novo_no  # Anterior último aponta para o novo
            self.tail = novo_no       # Atualizar tail
        self.size += 1
        return True

    def _inserir_posicao_interno(self, data: Any, posicao: int):
        """Método interno para inserir em posição específica"""
        if posicao < 0 or posicao > self.size:
            raise IndexError("Posição inválida")

        if posicao == 0:
            return self._inserir_inicio_interno(data)
        elif posicao == self.size:
            return self._inserir_fim_interno(data)

        novo_no = NoCircular(data)
        atual = self.head
        for _ in range(posicao - 1):
            atual = atual.next

        novo_no.next = atual.next
        atual.next = novo_no
        self.size += 1
        return True

    def _remover_inicio_interno(self):
        """Método interno para remover do início"""
        if self.head is None:
            raise IndexError("Lista vazia")

        data = self.head.data

        if self.size == 1:
            # Último elemento
            self.head = None
            self.tail = None
        else:
            # Remover do início e manter circularidade
            self.tail.next = self.head.next  # O último agora aponta para o segundo
            self.head = self.head.next       # O segundo vira o primeiro

        self.size -= 1
        return data

    def _remover_fim_interno(self):
        """Método interno para remover do fim"""
        if self.head is None:
            raise IndexError("Lista vazia")

        data = self.tail.data

        if self.size == 1:
            # Último elemento
            self.head = None
            self.tail = None
        else:
            # Encontrar o penúltimo nó
            atual = self.head
            while atual.next != self.tail:
                atual = atual.next

            # Remover do fim e manter circularidade
            atual.next = self.head  # Penúltimo agora aponta para o primeiro
            self.tail = atual       # Penúltimo vira o último

        self.size -= 1
        return data

    def _remover_posicao_interno(self, posicao: int):
        """Método interno para remover de posição específica"""
        if posicao < 0 or posicao >= self.size:
            raise IndexError("Posição inválida")

        if posicao == 0:
            return self._remover_inicio_interno()
        elif posicao == self.size - 1:
            return self._remover_fim_interno()

        atual = self.head
        for _ in range(posicao - 1):
            atual = atual.next

        data = atual.next.data
        atual.next = atual.next.next
        self.size -= 1
        return data

    def _buscar_interno(self, data: Any):
        """Método interno para buscar elemento (com proteção contra ciclo infinito)"""
        if self.head is None:
            return -1

        atual = self.head
        posicao = 0

        # Usar contador para evitar loop infinito
        for _ in range(self.size):
            if atual.data == data:
                return posicao
            atual = atual.next
            posicao += 1

        return -1

    def _listar_interno(self):
        """Método interno para listar todos os elementos (com proteção contra ciclo infinito)"""
        if self.head is None:
            return []

        elementos = []
        atual = self.head

        # Usar contador para evitar loop infinito
        for _ in range(self.size):
            elementos.append(atual.data)
            atual = atual.next

        return elementos

    def _percorrer_circular_interno(self, num_voltas: int = 2):
        """Método interno para percorrer a lista circular várias voltas completas"""
        if self.head is None:
            return []

        elementos = []
        atual = self.head
        total_elementos = self.size * num_voltas

        for _ in range(total_elementos):
            elementos.append(atual.data)
            atual = atual.next

        return elementos

    def verificar_circularidade(self):
        """Método interno para verificar se a lista está realmente circular"""
        if self.head is None:
            return True  # Lista vazia é considerada circular

        if self.size == 1:
            return self.head.next == self.head

        # Verificar se o tail aponta para head
        return self.tail.next == self.head

    # Métodos públicos com medição de tempo
    def inserir_inicio(self, data: Any):
        """Insere elemento no início da lista"""
        return self._medir_tempo("inserir_inicio", self._inserir_inicio_interno, data)

    def inserir_fim(self, data: Any):
        """Insere elemento no fim da lista"""
        return self._medir_tempo("inserir_fim", self._inserir_fim_interno, data)

    def inserir_posicao(self, data: Any, posicao: int):
        """Insere elemento em posição específica"""
        return self._medir_tempo("inserir_posicao", self._inserir_posicao_interno, data, posicao)

    def remover_inicio(self):
        """Remove elemento do início da lista"""
        return self._medir_tempo("remover_inicio", self._remover_inicio_interno)

    def remover_fim(self):
        """Remove elemento do fim da lista"""
        return self._medir_tempo("remover_fim", self._remover_fim_interno)

    def remover_posicao(self, posicao: int):
        """Remove elemento de posição específica"""
        return self._medir_tempo("remover_posicao", self._remover_posicao_interno, posicao)

    def buscar(self, data: Any):
        """Busca elemento na lista e retorna sua posição"""
        return self._medir_tempo("buscar", self._buscar_interno, data)

    def listar(self):
        """Lista todos os elementos da lista"""
        return self._medir_tempo("listar", self._listar_interno)

    def percorrer_circular(self, num_voltas: int = 2):
        """Percorre a lista circular várias voltas (funcionalidade exclusiva)"""
        return self._medir_tempo("percorrer_circular", self._percorrer_circular_interno, num_voltas)

    def verificar_circularidade_com_tempo(self):
        """Verifica se a lista está realmente circular com medição de tempo (funcionalidade exclusiva)"""
        return self._medir_tempo("verificar_circularidade", self.verificar_circularidade)

    def obter_tamanho(self):
        """Retorna o tamanho atual da lista"""
        return self.size

    def esta_vazia(self):
        """Verifica se a lista está vazia"""
        return self.size == 0

    def limpar(self):
        """Remove todos os elementos da lista"""
        def _limpar_interno():
            self.head = None
            self.tail = None
            self.size = 0
            return True

        return self._medir_tempo("limpar", _limpar_interno)

    def exibir_estatisticas(self):
        """Exibe estatísticas dos tempos de execução"""
        if not self.tempos_execucao:
            print("Nenhuma operação foi executada ainda.")
            return

        operacoes = {}
        for registro in self.tempos_execucao:
            op = registro['operacao']
            if op not in operacoes:
                operacoes[op] = []
            operacoes[op].append(registro['tempo_execucao_ms'])

        print("\n=== ESTATÍSTICAS DE DESEMPENHO (LISTA CIRCULAR) ===")
        print(f"Sistema Operacional: {platform.system()} {platform.release()}")
        print(f"Arquitetura: {platform.machine()}")
        print(f"Total de operações executadas: {len(self.tempos_execucao)}")
        print("\nTempos médios por operação:")

        for operacao, tempos in operacoes.items():
            tempo_medio = sum(tempos) / len(tempos)
            tempo_min = min(tempos)
            tempo_max = max(tempos)
            print(f"  {operacao}: {tempo_medio:.4f} ms (min: {tempo_min:.4f}, max: {tempo_max:.4f}, execuções: {len(tempos)})")

In [3]:
def gerar_dataset_performance_circular():
    """
    Gera um dataset completo executando N operações de cada tipo
    com números aleatórios para comparação entre plataformas
    """

    N = 2_000

    print("=== GERAÇÃO DO DATASET DE PERFORMANCE (LISTA CIRCULAR) ===")
    print(f"Sistema: {platform.system()} {platform.release()}")
    print(f"Arquitetura: {platform.machine()}")
    print(f"Executando {N} operações de cada tipo...")
    print("Aguarde, isso pode levar alguns minutos...\n")

    lista = ListaCircular()

    # Configurar seed para reprodutibilidade (opcional)
    random.seed(42)

    # 1. INSERÇÕES NO INÍCIO - N números
    print("Executando inserções no início...")
    for i in range(N):
        numero_aleatorio = random.randint(1, N)
        lista.inserir_inicio(numero_aleatorio)

        # Mostrar progresso a cada int(N/10) operações
        if (i + 1) % int(N/10) == 0:
            print(f"  Inserções início: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após inserções no início: {lista.obter_tamanho():,}")
    print(f"Lista circular válida: {lista.verificar_circularidade()}\n")

    # 2. INSERÇÕES NO FIM - N números
    print("Executando inserções no fim...")
    for i in range(N):
        numero_aleatorio = random.randint(1, N)
        lista.inserir_fim(numero_aleatorio)

        if (i + 1) % int(N/10) == 0:
            print(f"  Inserções fim: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após inserções no fim: {lista.obter_tamanho():,}")
    print(f"Lista circular válida: {lista.verificar_circularidade()}\n")

    # 3. BUSCAS - N números
    print("Executando buscas...")
    for i in range(N):
        # Buscar números aleatórios (alguns existem, outros não)
        numero_aleatorio = random.randint(1, N*2)  # Range maior para incluir elementos não existentes
        lista.buscar(numero_aleatorio)

        if (i + 1) % int(N/10) == 0:
            print(f"  Buscas: {i + 1:,}/{N} concluídas")

    print("Buscas concluídas\n")

    # 4. PERCORRER CIRCULAR - N/10 operações (nova funcionalidade)
    #print("Executando percorrimentos circulares...")
    #for i in range(int(N/10)):
    #    num_voltas = 3  # 3 voltas
    #    lista.percorrer_circular(num_voltas)

    #    if (i + 1) % 20 == 0:
    #        print(f"  Percorrimentos circulares: {i + 1}/{int(N/10)} concluídos")

    #print("Percorrimentos circulares concluídos\n")

    # 5. VERIFICAÇÕES DE CIRCULARIDADE - N/10 operações (nova funcionalidade)
    #print("Executando verificações de circularidade...")
    #for i in range(int(N/10)):
    #    lista.verificar_circularidade()

    #    if (i + 1) % 20 == 0:
    #        print(f"  Verificações circularidade: {i + 1}/{int(N/10)} concluídas")

    #print("Verificações de circularidade concluídas\n")

    # 6. INSERÇÕES EM POSIÇÕES ALEATÓRIAS - N números
    print("Executando inserções em posições aleatórias...")
    for i in range(N):
        numero_aleatorio = random.randint(1, N)
        # Posição aleatória válida (entre 0 e tamanho atual)
        posicao_aleatoria = random.randint(0, lista.obter_tamanho())
        try:
            lista.inserir_posicao(numero_aleatorio, posicao_aleatoria)
        except IndexError:
            # Se por algum motivo a posição for inválida, inserir no fim
            lista.inserir_fim(numero_aleatorio)

        if (i + 1) % int(N/10) == 0:
            print(f"  Inserções posição: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após inserções por posição: {lista.obter_tamanho():,}")
    print(f"Lista circular válida: {lista.verificar_circularidade()}\n")

    # 7. REMOÇÕES DO INÍCIO - N números
    print("Executando remoções do início...")
    for i in range(N):
        if not lista.esta_vazia():
            lista.remover_inicio()
        else:
            # Se a lista estiver vazia, adicionar um elemento e remover
            lista.inserir_inicio(random.randint(1, int(N/10)))
            lista.remover_inicio()

        if (i + 1) % int(N/10) == 0:
            print(f"  Remoções início: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após remoções do início: {lista.obter_tamanho():,}")
    if not lista.esta_vazia():
        print(f"Lista circular válida: {lista.verificar_circularidade()}\n")
    else:
        print("Lista vazia após remoções\n")

    # 8. REMOÇÕES DO FIM - N números
    print("Executando remoções do fim...")
    for i in range(N):
        if not lista.esta_vazia():
            lista.remover_fim()
        else:
            # Se a lista estiver vazia, adicionar um elemento e remover
            lista.inserir_fim(random.randint(1, int(N/10)))
            lista.remover_fim()

        if (i + 1) % int(N/10) == 0:
            print(f"  Remoções fim: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após remoções do fim: {lista.obter_tamanho():,}")
    if not lista.esta_vazia():
        print(f"Lista circular válida: {lista.verificar_circularidade()}\n")
    else:
        print("Lista vazia após remoções\n")

    # 9. REMOÇÕES DE POSIÇÕES ALEATÓRIAS - N números
    print("Executando remoções de posições aleatórias...")

    # Garantir que temos elementos suficientes para remover
    if lista.obter_tamanho() < int(N/2):
        print("  Adicionando elementos para garantir remoções...")
        for _ in range(int(N/2)):
            lista.inserir_fim(random.randint(1, N))

    for i in range(N):
        if lista.obter_tamanho() > 0:
            posicao_aleatoria = random.randint(0, lista.obter_tamanho() - 1)
            try:
                lista.remover_posicao(posicao_aleatoria)
            except IndexError:
                # Se por algum motivo a posição for inválida, remover do início
                if not lista.esta_vazia():
                    lista.remover_inicio()
        else:
            # Se a lista estiver vazia, adicionar e remover
            lista.inserir_inicio(random.randint(1, int(N/10)))
            lista.remover_posicao(0)

        if (i + 1) % int(N/10) == 0:
            print(f"  Remoções posição: {i + 1:,}/{N} concluídas")

    print(f"Tamanho da lista após remoções por posição: {lista.obter_tamanho():,}")
    if not lista.esta_vazia():
        print(f"Lista circular válida: {lista.verificar_circularidade()}\n")
    else:
        print("Lista vazia após remoções\n")

    # 10. LISTAGENS - N/10 operações (reduzidas pois são pesadas)
    #print("Executando listagens...")

    # Adicionar alguns elementos para as listagens
    #if lista.esta_vazia():
    #    for _ in range(100):
    #        lista.inserir_fim(random.randint(1, 100))

    #for i in range(int(N/10)):
    #    lista.listar()
    #    if i % 20 == 0:
    #        print(f"  Listagens: {i + 1}/{int(N/10)} concluídas")

    #print("Listagens concluídas\n")

    print(f"Tamanho final da lista: {lista.obter_tamanho():,}")
    if not lista.esta_vazia():
        print(f"Lista circular final válida: {lista.verificar_circularidade()}")

    # Relatório final
    total_operacoes = len(lista.tempos_execucao)
    print("\n=== RELATÓRIO FINAL (LISTA CIRCULAR) ===")
    print(f"Total de operações executadas: {total_operacoes:,}")
    print(f"Sistema operacional: {platform.system()}")
    print(f"Versão do SO: {platform.release()}")
    print(f"Arquitetura: {platform.machine()}")

    # Contar operações por tipo
    contagem_operacoes = {}
    for registro in lista.tempos_execucao:
        operacao = registro['operacao']
        contagem_operacoes[operacao] = contagem_operacoes.get(operacao, 0) + 1

    print("\nDistribuição das operações:")
    for operacao, count in sorted(contagem_operacoes.items()):
        print(f"  {operacao}: {count:,} operações")

    print(f"\nDados salvos em: tempos_de_execucao/lista_encadeada_circular_{platform.system().lower()}_tempos_execucao.json")
    print("Dataset gerado com sucesso!")

    return lista

# Executar geração do dataset
if __name__ == "__main__":
    lista_dataset = gerar_dataset_performance_circular()

=== GERAÇÃO DO DATASET DE PERFORMANCE (LISTA CIRCULAR) ===
Sistema: Linux 6.14.0-35-generic
Arquitetura: x86_64
Executando 2000 operações de cada tipo...
Aguarde, isso pode levar alguns minutos...

Executando inserções no início...
  Inserções início: 200/2000 concluídas
  Inserções início: 400/2000 concluídas
  Inserções início: 600/2000 concluídas
  Inserções início: 800/2000 concluídas
  Inserções início: 1,000/2000 concluídas
  Inserções início: 1,200/2000 concluídas
  Inserções início: 1,400/2000 concluídas
  Inserções início: 1,600/2000 concluídas
  Inserções início: 1,800/2000 concluídas
  Inserções início: 2,000/2000 concluídas
Tamanho da lista após inserções no início: 2,000
Lista circular válida: True

Executando inserções no fim...
  Inserções fim: 200/2000 concluídas
  Inserções fim: 400/2000 concluídas
  Inserções fim: 600/2000 concluídas
  Inserções fim: 800/2000 concluídas
  Inserções fim: 1,000/2000 concluídas
  Inserções fim: 1,200/2000 concluídas
  Inserções fim: 1,40